In [1]:
import csv
import math
import ipywidgets as widgets
from IPython.display import display, clear_output


In [2]:
def read_student_data(file_path="records.csv"): # opens file called records.csv -- obselete do not use this function
    records = [] #generates empty list
    try:
        with open(file_path, mode='r', newline='') as file:
            reader = csv.DictReader(file)
            for row in reader: 
                row['CGPA'] = float(row['CGPA'])#changes cgpa data type from str to flt, so we can iterate on it
                records.append(row)#puts the list into the records list, creating list of lists
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")#handles situation where file is gone
        return None
    return records#this function can be replaced by the json loader

function obselete

In [3]:
def students_by_tg(student_data): #reads the output list of list from the previous function
    tutorial_groups = {} #empty dict
    for student in student_data:
        tg = student["Tutorial Group"] #initialising tg variable to the tg of indv students
        if tg not in tutorial_groups: 
            tutorial_groups[tg] = [] #creates new tg if not already present
        tutorial_groups[tg].append(student) #adds the student to their tg
    return tutorial_groups

function obselete

In [4]:
def snake_draft(students_sorted,num_teams=10):#this function creates a WIP grouping for an individual tg based solely on gpa in a snake draft format
    def _draft(students_sorted, num_teams=10): 
        teams = [[] for tm in range(num_teams)] # generates an empty list for each team
    
        current_team_index = 0
        direction = 1

        for student in students_sorted:
            teams[current_team_index].append(student) #adds the student at current index to the current group(ungrouped student with the lowest index-highest gpa)
            current_team_index += direction

            if current_team_index == num_teams: #block to reverse direction, creating the snake draft
                direction = -1
                current_team_index = num_teams - 1
            elif current_team_index == -1:
                direction = 1
                current_team_index = 0
            
        return teams
    
    if isinstance(students_sorted,dict):
        return {tg: _draft(students ,num_teams) for tg,students in students_sorted.items()}
    else:   
        return _draft(students_sorted, num_teams)

In [ ]:
def diversity_score(team): #grades how diverse each group is(lower better)
    score = 0
    gender_counts = {}
    school_counts = {}

    for student in team:
        gender_counts[student['Gender']] = gender_counts.get(student['Gender'], 0) + 1
        school_counts[student['School']] = school_counts.get(student['School'], 0) + 1

    for count in gender_counts.values():
        if count >= 3:
            score += (count - 2) * 10 #every 3rd person onwards of the same gender will add 10 to the diversity score

    for count in school_counts.values():
        if count >= 2:
            score += (count - 2) * 10 #every 2nd person onwards of the same school will add 10 to the diversity score

    return score


In [6]:
def total_cgpa_variance(teams):
    if not teams or not all(teams):
        return 0
    
    team_averages = [sum(s['CGPA'] for s in team) / len(team) for team in teams] #calculate the average cgpa of each group
    overall_average = sum(team_averages) / len(team_averages) #calculate the average cgpa of the tg
    
    variance = sum((avg - overall_average) ** 2 for avg in team_averages) / len(team_averages) #calculates the variance of the average tg (standard s^2 variance)
    return variance

In [7]:
def optimize_teams(teams_in_tg, diversity_weight=0.70, cgpa_weight=0.30):
    DIVERSITY_WEIGHT = diversity_weight
    CGPA_WEIGHT = cgpa_weight #set weights

    for k in range(1000): #number of tests to run
        current_diversity_score = sum(diversity_score(t) for t in teams_in_tg)
        current_cgpa_variance = total_cgpa_variance(teams_in_tg) #calculates initial score and variance of current teams

        if current_diversity_score == 0 and current_cgpa_variance < 0.001: #if teams are new perfect, dont change
            break 

        best_improvement_score = 0
        best_swap_details = None #instansiation of vars

        for i in range(len(teams_in_tg)):
            for j in range(i + 1, len(teams_in_tg)): #every combination of teams

                for s1_idx in range(len(teams_in_tg[i])):
                    for s2_idx in range(len(teams_in_tg[j])): 
                        team1 = teams_in_tg[i]
                        team2 = teams_in_tg[j]#every combination of students within the chosen teams

                        team1[s1_idx], team2[s2_idx] = team2[s2_idx], team1[s1_idx]#simulate the swap

                        new_diversity_score = sum(diversity_score(t) for t in teams_in_tg)
                        new_cgpa_variance = total_cgpa_variance(teams_in_tg) #new scores for the swap
                        
                        diversity_improvement = current_diversity_score - new_diversity_score #change in diversity score
                        cgpa_improvement = current_cgpa_variance - new_cgpa_variance #change in cgpa variance

                        total_improvement = (diversity_improvement * DIVERSITY_WEIGHT) + (cgpa_improvement * CGPA_WEIGHT)

                        if total_improvement > best_improvement_score:
                            best_improvement_score = total_improvement
                            best_swap_details = (i, j, s1_idx, s2_idx)#if this is the best swap, store it 

                        team1[s1_idx], team2[s2_idx] = team2[s2_idx], team1[s1_idx] #swap back for now to check all other possibilities
        
        if best_swap_details:
            i, j, s1, s2 = best_swap_details
            teams_in_tg[i][s1], teams_in_tg[j][s2] = teams_in_tg[j][s2], teams_in_tg[i][s1]#sets the best swap after checking all
        else:
            break#if there are no good swaps, break the loop to save computational power
            
    return teams_in_tg

In [8]:
def create_teams_by_size(students_sorted, team_size):
    if isinstance(students_sorted, dict):
        return {tg: create_teams_by_size(students, team_size) for tg, students in students_sorted.items()}
    n = len(students_sorted)
    if n == 0:
        return []
    num_teams = math.ceil(n / team_size)#returns number of teams with team size.
    return snake_draft(students_sorted, num_teams=num_teams) #reuse snake


In [9]:
# GUI: sliders to pick team size (students per team) and the two weights.
def _run_all(team_size, diversity_wt, cgpa_wt):
    # normalize weights so they sum to 1 (if both zero, fallback to defaults)
    total = diversity_wt + cgpa_wt
    if total == 0:
        diversity_wt, cgpa_wt = 0.7, 0.3
    else:
        diversity_wt, cgpa_wt = diversity_wt / total, cgpa_wt / total

    data = read_student_data("records.csv")
    if not data:
        print("No data found. Ensure records.csv is present or use a JSON loader.")
        return

    tutorial_groups = students_by_tg(data)
    all_optimized_teams = {}

    for tg_name, students in tutorial_groups.items():
        print(f"Processing Tutorial Group: {tg_name} (team size={team_size})...")
        initial_teams = create_teams_by_size(students, team_size)
        optimized_teams = optimize_teams(initial_teams, diversity_weight=diversity_wt, cgpa_weight=cgpa_wt)
        all_optimized_teams[tg_name] = optimized_teams

    final_list = data_compilation(all_optimized_teams)
    write_output_csv(final_list, output_path="final_teams.csv")
    print("Done. Output written to final_teams.csv.")

# Widgets
team_size_slider = widgets.IntSlider(value=10, min=4, max=10, step=1, description='Team Size')
diversity_slider = widgets.FloatSlider(value=0.7, min=0.0, max=1.0, step=0.01, description='Diversity Wt')
cgpa_slider = widgets.FloatSlider(value=0.3, min=0.0, max=1.0, step=0.01, description='CGPA Wt')
run_button = widgets.Button(description='Run')
output_area = widgets.Output()

def on_run_clicked(b):
    with output_area:
        clear_output(wait=True)
        _run_all(team_size_slider.value, diversity_slider.value, cgpa_slider.value)

run_button.on_click(on_run_clicked)
display(widgets.HBox([team_size_slider, diversity_slider, cgpa_slider, run_button]))
display(output_area)


Output()

In [10]:
def data_compilation(all_optimized_tgs): #prepares data for csv writing and assigns team numbers
    final_student_list = []
    sorted_tg_names = sorted(all_optimized_tgs.keys())

    for tg_name in sorted_tg_names:
        list_of_teams = all_optimized_tgs[tg_name]#grabs the list of teams in a tg
        for team_index, team in enumerate(list_of_teams):
            team_number = team_index + 1 # adds a group number 
            for student in team:
                student["Team Assigned"] = team_number #assigns the group number to the student
                final_student_list.append(student) #adds the student, with all their data into the list as a list
    return final_student_list

In [11]:
def write_output_csv(final_student_list, output_path="final_teams.csv"):
    if not final_student_list:
        print("No data available to write.") #incase of faulty inputs
        return
    
    headers = ["Tutorial Group", "Team Assigned", "Student ID", "Name", "School", "Gender", "CGPA"] #initialise the headers
    
    output_data = [{header: student.get(header) for header in headers} for student in final_student_list]

    with open(output_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(output_data) #write file
    print(f"Successfully created GPA-optimized allocation file: {output_path}")


In [12]:
data = read_student_data("records.csv")
if not data:
    print("No data found. Ensure records.csv is present or use a JSON loader.")
else:
    tutorial_groups = students_by_tg(data)
    all_optimized_teams = {}

    TEAM_SIZE = 10  # students per team (4-10 supported in GUI)
    DIVERSITY_WT = 0.7
    CGPA_WT = 0.3

    for tg_name, students in tutorial_groups.items():
        print(f"Processing Tutorial Group: {tg_name} (team size={TEAM_SIZE})...")
        initial_teams = create_teams_by_size(students, TEAM_SIZE)
        optimized_teams = optimize_teams(initial_teams, diversity_weight=DIVERSITY_WT, cgpa_weight=CGPA_WT)
        all_optimized_teams[tg_name] = optimized_teams

    final_list = data_compilation(all_optimized_teams)
    write_output_csv(final_list)
    print("Done. Output written to final_teams.csv")


Processing Tutorial Group: G-1 (team size=10)...


TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'